# Classification Algorithms
#### By Zachary Austin Ellis & Jose Carlos Gomez-Vazquez

In this notebook we will implement our own version of the Decision Tree Classifier, Random Forest Classifer, and Naive Bayes Classifier then compare their performance against SciKit-Learn's implementations.
For these algorithms, the Red Wine Quality dataset provided by Kaggle will be used.

Source: https://www.kaggle.com/uciml/red-wine-quality-cortez-et-al-2009

In [2]:
import numpy as np
import pandas as pd

# Used to compare our implementation's performance
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB

### Preprocessing

In [17]:
def train_test_split(df, test_size):
    testing_set = df.sample(test_size)
    df.drop(index=testing_set.index, inplace=True)
    df.reset_index(drop=True, inplace=True)
    
    X_train = df.drop(columns="quality")
    X_test = testing_set.drop(columns="quality").reset_index(drop=True)
    
    Y_train = df["quality"]
    Y_test = testing_set["quality"].reset_index(drop=True)
    
    return X_train, X_test, Y_train, Y_test


wine_data = pd.read_csv("../data/winequality-red.csv")
print("\t\t\t\t\tRed Wine Data\n", wine_data)

X_train, X_test, Y_train, Y_test = train_test_split(wine_data.copy(), 100)

					Red Wine Data
    fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0            7.4              0.70         0.00             1.9      0.076   
1            7.8              0.88         0.00             2.6      0.098   
2            7.8              0.76         0.04             2.3      0.092   
3           11.2              0.28         0.56             1.9      0.075   
4            7.4              0.70         0.00             1.9      0.076   

   free sulfur dioxide  total sulfur dioxide  density    pH  sulphates  \
0                 11.0                  34.0   0.9978  3.51       0.56   
1                 25.0                  67.0   0.9968  3.20       0.68   
2                 15.0                  54.0   0.9970  3.26       0.65   
3                 17.0                  60.0   0.9980  3.16       0.58   
4                 11.0                  34.0   0.9978  3.51       0.56   

   alcohol  quality  
0      9.4        5  
1      9.8        5  


### Decision Tree Classifier

### Decision Tree Comparision

### Random Forest Classifier

### Random Forest Comparison

### Naive Bayes Classifier

### Naive Bayes Comparison